# Load PointClouds in Spark using PDAL
We calculate the Normalized Height for each point.

In [41]:
import geotrellis.pointcloud.spark.Extent3D
import geotrellis.pointcloud.spark.io.hadoop.HadoopPointCloudRDD
import geotrellis.pointcloud.spark.tiling.CutPointCloud
import geotrellis.raster.GridExtent
import geotrellis.spark.SpatialKey
import geotrellis.spark.io.hadoop.HadoopCollectionLayerReader
import geotrellis.spark.tiling.LayoutDefinition
import geotrellis.vector.Extent
import io.pdal.Pipeline
import io.pdal.pipeline.{EigenValuesFilter, LasRead, PythonFilter, Read}
import org.apache.hadoop.fs.Path
import org.apache.spark.rdd.RDD
import org.apache.spark.{SparkConf, SparkContext}
import spire.syntax.cfor._

In [2]:
implicit val sparkContext = sc

Waiting for a Spark session to start...

sparkContext = org.apache.spark.SparkContext@422c0cfb


Spark app-20180507100359-0000: Some(http://145.100.58.104:4040)

### Read HDFS and apply HagFilter

In [55]:
val laz_path = new Path("/user/hadoop/ahn3/C_25EZ2.las")
val pipelineExpr = Read("local") ~ HagFilter()
val rdd_laz = HadoopPointCloudRDD(laz_path, options = HadoopPointCloudRDD.Options(pipeline = pipelineExpr))

laz_path = /user/hadoop/ahn3/C_25EZ2.las
pipelineExpr = List(Read(local,None,None,None), HagFilter(filters.hag))
rdd_laz = NewHadoopRDD[56] at newAPIHadoopRDD at HadoopPointCloudRDD.scala:76


NewHadoopRDD[56] at newAPIHadoopRDD at HadoopPointCloudRDD.scala:76

### Verify if HeightAboveGround shows up in the schema

In [48]:
rdd_laz.cache()
rdd_laz.map{ case (h, i) => h.schema}.take(1)

Array("{
  "schema":
  {
    "dimensions":
    [
      {
        "name": "X",
        "size": 8,
        "type": "floating"
      },
      {
        "name": "Y",
        "size": 8,
        "type": "floating"
      },
      {
        "name": "Z",
        "size": 8,
        "type": "floating"
      },
      {
        "name": "Intensity",
        "size": 2,
        "type": "unsigned"
      },
      {
        "name": "ReturnNumber",
        "size": 1,
        "type": "unsigned"
      },
      {
        "name": "NumberOfReturns",
        "size": 1,
        "type": "unsigned"
      },
      {
        "name": "ScanDirectionFlag",
        "size": 1,
        "type": "unsigned"
      },
      {
        "name": "EdgeOfFlightLine",
        "size": 1,
        "type": "unsigne...


[{
  "schema":
  {
    "dimensions":
    [
      {
        "name": "X",
        "size": 8,
        "type": "floating"
      },
      {
        "name": "Y",
        "size": 8,
        "type": "floating"
      },
      {
        "name": "Z",
        "size": 8,
        "type": "floating"
      },
      {
        "name": "Intensity",
        "size": 2,
        "type": "unsigned"
      },
      {
        "name": "ReturnNumber",
        "size": 1,
        "type": "unsigned"
      },
      {
        "name": "NumberOfReturns",
        "size": 1,
        "type": "unsigned"
      },
      {
        "name": "ScanDirectionFlag",
        "size": 1,
        "type": "unsigned"
      },
      {
        "name": "EdgeOfFlightLine",
        "size": 1,
        "type": "unsigned"
      },
      {
        "name": "Classification",
        "size": 1,
        "type": "unsigned"
      },
      {
        "name": "ScanAngleRank",
        "size": 4,
        "type": "floating"
      },
      {
        "name": "Use

### Load all the points into RDD and compare Z with NormalizedHeight

In [65]:
val points :RDD[(Double, Double, Double, Byte, Double)] = rdd_laz.flatMap(_._2).mapPartitions{ _.map { packedPoints =>
    var res = new Array[(Double, Double, Double, Byte, Double)](packedPoints.length)
    cfor(0)(_ < packedPoints.length, _ + 1) { i =>
        res(i) = (packedPoints.getX(i), packedPoints.getY(i), packedPoints.getZ(i), packedPoints.getByte(i, dim = "Classification"), packedPoints.getDouble(i, dim = "HeightAboveGround"))
    }
    res
}}.flatMap( m => m).filter(_._4 != 2)//.filter(_._3 > 12)

points = MapPartitionsRDD[82] at filter at <console>:95


MapPartitionsRDD[82] at filter at <console>:95

In [62]:
points.count()

15785689

In [79]:
points.sortBy{ case(x, y, z, c, zh) => (x,y)}.take(10)

[(125000.0,487503.61100000003,10.738,6,7.497999999999999), (125000.0,487512.52400000003,10.867,6,7.644000000000001), (125000.0,487522.228,10.901,6,9.167), (125000.0,488181.435,-0.47500000000000003,9,-0.8190000000000001), (125000.0,488317.335,-0.466,9,-0.020000000000000018), (125000.0,488351.775,-0.47400000000000003,9,-0.028000000000000025), (125000.0,488396.354,-0.47900000000000004,9,-0.03300000000000003), (125000.0,488475.981,-0.481,9,-0.08999999999999997), (125000.0,488810.282,8.955,1,8.016), (125000.0,488861.316,4.473,1,3.635)]

## Tile the pointCloud

In [58]:
val extent = Extent(0, 0, 10, 5)
val gridExtent = GridExtent(extent, 1, 1)  // 10×5 pixels
val layoutDefinition = LayoutDefinition(gridExtent, 10, 5)
val pointCloud = rdd_laz.flatMap(_._2)
val pointCloudTiled = CutPointCloud(pointCloud, layoutDefinition)

extent = Extent(0.0, 0.0, 10.0, 5.0)
gridExtent = GridExtent(Extent(0.0, 0.0, 10.0, 5.0),1.0,1.0)
layoutDefinition = GridExtent(Extent(0.0, 0.0, 10.0, 5.0),1.0,1.0)
pointCloud = MapPartitionsRDD[62] at flatMap at <console>:103
pointCloudTiled = ContextRDD[65] at RDD at ContextRDD.scala:35


ContextRDD[65] at RDD at ContextRDD.scala:35

In [67]:
val points2 :RDD[(SpatialKey, Array[(Double, Double, Double, Byte, Double)])] = pointCloudTiled.mapPartitions{ _.map { case (s, packedPoints) => (s,{
  var res = new Array[(Double, Double, Double, Byte, Double)](packedPoints.length)
  cfor(0)(_ < packedPoints.length, _ + 1) { i =>
    res(i) = (packedPoints.getX(i), packedPoints.getY(i), packedPoints.getZ(i), packedPoints.getByte(i, dim = "Classification"), packedPoints.getDouble(i, dim = "HeightAboveGround"))
  }
  res})
}}
val pointsb = points2.flatMap(m => m._2).filter(_._4 != 2)

[Stage 41:====================================================>(993 + 7) / 1000]

points2 = MapPartitionsRDD[83] at mapPartitions at <console>:97
pointsb = MapPartitionsRDD[88] at filter at <console>:104


MapPartitionsRDD[88] at filter at <console>:104

In [60]:
pointsb.count()

[Stage 32:====================================================>(995 + 5) / 1000]

15785689

In [80]:
pointsb.sortBy{ case(x, y, z, c, zh) => (x,y)}.take(10)

[Stage 81:====================================================>(996 + 4) / 1000]

[(125000.0,487503.61100000003,10.738,6,7.497999999999999), (125000.0,487512.52400000003,10.867,6,7.644000000000001), (125000.0,487522.228,10.901,6,9.167), (125000.0,488181.435,-0.47500000000000003,9,-0.8190000000000001), (125000.0,488317.335,-0.466,9,-0.020000000000000018), (125000.0,488351.775,-0.47400000000000003,9,-0.028000000000000025), (125000.0,488396.354,-0.47900000000000004,9,-0.03300000000000003), (125000.0,488475.981,-0.481,9,-0.08999999999999997), (125000.0,488810.282,8.955,1,8.016), (125000.0,488861.316,4.473,1,3.635)]